In [1]:
import numpy as np
import random
from keras.models import Model, load_model
from keras.layers import Dense, Input
from keras.layers import LSTM
import string
import h5py
import gc

C:\Users\Dwarkanath\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
chars = string.printable
vocab_size = len(chars)
char_to_ix = {ch:i for i,ch in enumerate(chars)}
ix_to_char = {i:ch for i,ch in enumerate(chars)}
print(ix_to_char)

{0: '0', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5', 6: '6', 7: '7', 8: '8', 9: '9', 10: 'a', 11: 'b', 12: 'c', 13: 'd', 14: 'e', 15: 'f', 16: 'g', 17: 'h', 18: 'i', 19: 'j', 20: 'k', 21: 'l', 22: 'm', 23: 'n', 24: 'o', 25: 'p', 26: 'q', 27: 'r', 28: 's', 29: 't', 30: 'u', 31: 'v', 32: 'w', 33: 'x', 34: 'y', 35: 'z', 36: 'A', 37: 'B', 38: 'C', 39: 'D', 40: 'E', 41: 'F', 42: 'G', 43: 'H', 44: 'I', 45: 'J', 46: 'K', 47: 'L', 48: 'M', 49: 'N', 50: 'O', 51: 'P', 52: 'Q', 53: 'R', 54: 'S', 55: 'T', 56: 'U', 57: 'V', 58: 'W', 59: 'X', 60: 'Y', 61: 'Z', 62: '!', 63: '"', 64: '#', 65: '$', 66: '%', 67: '&', 68: "'", 69: '(', 70: ')', 71: '*', 72: '+', 73: ',', 74: '-', 75: '.', 76: '/', 77: ':', 78: ';', 79: '<', 80: '=', 81: '>', 82: '?', 83: '@', 84: '[', 85: '\\', 86: ']', 87: '^', 88: '_', 89: '`', 90: '{', 91: '|', 92: '}', 93: '~', 94: ' ', 95: '\t', 96: '\n', 97: '\r', 98: '\x0b', 99: '\x0c'}


In [3]:
def get_text(filename):
    with open(filename, 'r',encoding = 'utf8') as f:
        text = f.read()
    return text

In [4]:
text = get_text('tweets.txt')

In [5]:
print(text[:1000])

Ted Mosby https://t.co/E91LykbNIe
What even was that performance. We're a long ball team now?
@okmanideep @abhi_manjunath Someone give them lasagna, dammit.
WTF is going on with Manchester City?
RT @hotjamz: he's makin a list
it's kept in plain text
an elf just got phished
you know what is next
santa claus has leaked 2.2 billion use…
RT @cantdrawarsenal: Happy Birthday Ramsey! https://t.co/l8xg4WsaD4
Everytime I call a restaurant, they say, "Yes, ma'am" Er...
@OdehEveryday This was in CStat?
@discopiggu Doesn't matter as long as the reader knows what it means.
People who grew up in the 90s now have purchasing power. So stuff that was popular then can be used to sell stuff now. That explains all the remakes of 90s songs and Google hiring Macaulay Culkin for their ad targeted at this age group.

What other one hit wonders will return. 🤔
PayTM recently did it with Kerala floods. Matching donations cost them a total of Rs. 1 crore . They collected over 30 crore from 12 lac users. If say 1%

In [6]:
def get_data_idx(text):
    X = []
    for i in text:
        try:
            X.append(char_to_ix[i])
        except KeyError:
            X.append(99)
    return X

In [7]:
X = get_data_idx(text[:20])
Y = char_to_ix[text[20]]

In [8]:
Tx = len(X)
Tx

20

In [9]:
y = np.zeros(vocab_size)

In [10]:
y[Y] =1
y

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [11]:
def convert_to_vec(X, vocab_size):
    Tx = len(X)
    x = np.zeros(shape = [Tx, vocab_size])
    for i, idx in enumerate(X):
        if idx is not None:
            x[i][idx] = 1
        else:
            continue
    return x

In [12]:
x = convert_to_vec(X, vocab_size)

In [13]:
x.shape

(20, 100)

In [14]:
def prepare_data(text, Tx, vocab_size, stride):
    m = len(text)
    x = np.zeros([m//stride, Tx, vocab_size])
    y = np.zeros([m//stride, vocab_size])
    for i in range(0, len(text)-Tx, stride):
        j = i//3
        X = get_data_idx(text[i:i+Tx])
        x[j] = convert_to_vec(X, vocab_size)
        try:
            y[j, char_to_ix[text[i+Tx]]] = 1
        except KeyError:
            y[j,99] = 1
    return x, y

In [15]:
x, y = prepare_data(text[:300000], Tx, vocab_size, stride = 3)

In [16]:
import gc 
gc.collect()

0

In [17]:
x.shape

(100000, 20, 100)

In [18]:
def tweet_model(input_shape, vocab_size):
    Tx, vocab_size = input_shape
    char_vecs = Input(shape = input_shape)
    X = LSTM(units = 224, name = 'LSTM1')(char_vecs)
    X = Dense(units = vocab_size, activation ='softmax', name = 'Dense1')(X)
    model = Model(inputs = char_vecs, outputs = X, name = 'tweet_model')
    
    return model

In [26]:
model = tweet_model([Tx, vocab_size], vocab_size)

In [27]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20, 100)           0         
_________________________________________________________________
LSTM1 (LSTM)                 (None, 224)               291200    
_________________________________________________________________
Dense1 (Dense)               (None, 100)               22500     
Total params: 313,700
Trainable params: 313,700
Non-trainable params: 0
_________________________________________________________________


In [28]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [40]:
model.fit(x , y, batch_size=16, verbose = 1, epochs = 10)

Epoch 1/10
33333/33333 [==============================] - 76s 2ms/step - loss: 0.0351 - acc: 0.9905
Epoch 2/10
33333/33333 [==============================] - 77s 2ms/step - loss: 0.0330 - acc: 0.9908
Epoch 3/10
33333/33333 [==============================] - 71s 2ms/step - loss: 0.0313 - acc: 0.9911
Epoch 4/10
33333/33333 [==============================] - 70s 2ms/step - loss: 0.0297 - acc: 0.9914: 1s -
Epoch 5/10
33333/33333 [==============================] - 71s 2ms/step - loss: 0.0280 - acc: 0.9918: 3s - loss: - ETA:
Epoch 6/10
33333/33333 [==============================] - 69s 2ms/step - loss: 0.0263 - acc: 0.9922
Epoch 7/10
33333/33333 [==============================] - 77s 2ms/step - loss: 0.0244 - acc: 0.9927
Epoch 8/10
33333/33333 [==============================] - 74s 2ms/step - loss: 0.0226 - acc: 0.9932
Epoch 9/10
33333/33333 [==============================] - 79s 2ms/step - loss: 0.0206 - acc: 0.9937
Epoch 10/10
33333/33333 [==============================] - 79s 2ms/step - l

In [26]:
y_pred = model.predict(np.expand_dims(x[0],0))

In [27]:
y_pred.shape

(1, 100)

In [28]:
x[:2].shape

(2, 20, 100)

In [29]:
X = np.argmax(x[0:1],axis = -1)
X.shape

(1, 20)

In [30]:
X = np.argmax(x[0:1],axis = -1)
Y_pred = np.argmax(y_pred, axis = -1)
bs, Tx = X.shape
for i in range(bs):
    for j in range(Tx):
        print(ix_to_char[X[i][j]], end = '')
    print('-')
    print(ix_to_char[Y_pred[i]])
    print('---')

Ted Mosby https://t.-
c
---


In [19]:
def sample(preds):
    vocab_size = preds.shape[-1]
    out = np.random.choice(range(vocab_size), p = preds.ravel())
    return out

In [32]:
out = sample(y_pred)

In [33]:
ix_to_char[out]

'c'

In [20]:
def generate_output():
    generated = ''
    usr_input = input("Write the beginning of your tweet. Your input is: ")
    # zero pad the sentence to Tx characters.
    sentence = ('{0:0>' + str(Tx) + '}').format(usr_input).lower()
    generated += usr_input
    print(usr_input, end = '')
    for i in range(280):

        x_pred = np.zeros((1, Tx, vocab_size))

        for t, char in enumerate(sentence):
            if char != '0':
                x_pred[0, t, char_to_ix[char]] = 1.

        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds)
        next_char = ix_to_char[next_index]

        generated += next_char
        sentence = sentence[1:] + next_char

        print(next_char, end = '')
        if next_char == '\n':
            break

In [51]:
generate_output()

Write the beginning of your tweet. Your input is: queen
queen: AAst jate : @ambalyambay ak SHa*AS!


In [60]:
#model.save('tweet_model.h5')

In [18]:
model = load_model('tweet_model.h5')

In [27]:
model.fit(x , y, batch_size=16, verbose = 1, epochs = 5)

Epoch 1/5
100000/100000 [==============================] - 208s 2ms/step - loss: 0.0159 - acc: 0.9951
Epoch 2/5
100000/100000 [==============================] - 207s 2ms/step - loss: 0.0154 - acc: 0.9953
Epoch 3/5
100000/100000 [==============================] - 208s 2ms/step - loss: 0.0149 - acc: 0.9954
Epoch 4/5
100000/100000 [==============================] - ETA: 0s - loss: 0.0144 - acc: 0.995 - 183s 2ms/step - loss: 0.0144 - acc: 0.9955
Epoch 5/5
100000/100000 [==============================] - 193s 2ms/step - loss: 0.0140 - acc: 0.9956


In [28]:
generate_output()

Write the beginning of your tweet. Your input is: Modi
Modi curned andmervies winde"! Wharara Charzea -_- https://t.co/InMXyzvvQr


In [29]:
model.save('tweet_model.h5')